In [1]:
import os
import xarray as xr
from pangeo_forge_recipes.patterns import FilePattern, ConcatDim, MergeDim
import s3fs
from cmip6_downscaling.methods.common.utils import calc_auspicious_chunks_dict
import apache_beam as beam
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr

In [2]:
#parameters
model = "GISS-E2-1-G"
variable = "tas"

In [3]:
fs_s3 = s3fs.S3FileSystem(anon=True)
fp = f's3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1f2/{variable}/{variable}_day_{model}_historical_r1i1p1f2_gn_1950.nc'
f = fs_s3.open(fp, mode='rb')
ds = xr.open_dataset(f)

In [4]:
ds

<xarray.Dataset>
Dimensions:  (time: 365, lat: 600, lon: 1440)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 1950-12-31 12:00:00
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    tas      (time, lat, lon) float32 ...
Attributes: (12/23)
    downscalingModel:      BCSD
    activity:              NEX-GDDP-CMIP6
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    Conventions:           CF-1.7
    creation_date:         2021-10-04T18:41:40.796912+00:00
    frequency:             day
    ...                    ...
    history:               2021-10-04T18:41:40.796912+00:00: install global a...
    disclaimer:            This data is considered provisional and subject to...
    external_variables:    areacella
    cmip6_source_id:       GISS-E2-1-G
    cmip6_institution_id:  NASA-GISS
    cmip6_license:         CC-BY-SA 4.0

In [5]:
target_chunks = calc_auspicious_chunks_dict(ds['tas'], chunk_dims=('time',))

def format_function(time):
    return f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1f2/tas/tas_day_GISS-E2-1-G_historical_r1i1p1f2_gn_{time}.nc"

years = list(range(1950, 1952))
time_dim = ConcatDim("time", keys=years)

pattern = FilePattern(format_function, time_dim, file_type="netcdf4")
pattern = FilePattern.prune(pattern, nkeep=2)


In [6]:
bucket = 'nasa-eodc-data-store'
target_root = (
    f"s3://{bucket}/{model}"
    "zarr-target"
)
store_name = f"{variable}_day_{model}_historical_r1i1p1f1_gn.zarr"
target_store = os.path.join(target_root, store_name)


In [7]:
transforms = (
    beam.Create(pattern.items())
    | OpenURLWithFSSpec(open_kwargs={'anon': True})
    | OpenWithXarray(file_type=pattern.file_type)
    | StoreToZarr(
        store_name=store_name,
        target_root=target_root,
        combine_dims=pattern.combine_dim_keys,
        target_chunks=target_chunks,
    )
)

In [ ]:
with beam.Pipeline() as p:
    p | transforms

In [6]:
ds = xr.open_dataset(target_store, engine="zarr")
ds

type: KeyboardInterrupt [while running '[6]: Create|OpenURLWithFSSpec|OpenWithXarray|StoreToZarr/StoreToZarr/PrepareZarrTarget/Map(schema_to_zarr)']